In [2]:
from sumup import Sumup
from sumup.checkouts import CreateCheckoutBody, ProcessCheckoutBody
import uuid

import requests

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path = '.venv/.env')

True

In [3]:
SUMUP_API_KEY = os.getenv("sumup_api_key")
client = Sumup(api_key=SUMUP_API_KEY)
BASE_URL = "https://api.sumup.com"

In [4]:
# Get merchant profile
merchant = client.merchant.get()
merchant_code = merchant.merchant_profile.merchant_code
print(f"Merchant: {merchant_code}")

C:\Users\virgi\AppData\Local\Temp\ipykernel_9852\4117817524.py:2: DeprecationWarning: The _Retrieve a profile_ endpoint is deprecated, please use the `Merchant` object instead (see [Merchants](https://developer.sumup.com/api/merchants)).
  merchant = client.merchant.get()


Merchant: MNR0YQWH


In [5]:
def create_checkout(api_key: str, merchant_code: str, price: float, description: str = ""):
    """Création d'un checkout en sandbox (déjà fait chez toi, je garde la structure)."""
    client = Sumup(api_key=api_key)

    checkout = client.checkouts.create(
        body=CreateCheckoutBody(
            amount=price,
            currency="EUR",
            checkout_reference=str(uuid.uuid4()).split('-')[0],
            merchant_code=merchant_code,
            description=description,
            redirect_url="http://127.0.0.1:8000/paiement/resultat/",
            return_url="http://127.0.0.1:8000/webhook/"
        )
    )

    print("\n=== CHECKOUT CRÉÉ ===")
    print(checkout)

    return checkout


def simulate_payment(checkout_id: str):
    """Simule un paiement sandbox pour transformer un checkout en transaction."""
    url = f"{BASE_URL}/v0.1/checkouts/{checkout_id}/simulate"
    headers = {"Authorization": f"Bearer {SUMUP_API_KEY}"}

    response = requests.post(url, headers=headers, json={"status": "PAID"})
    response.raise_for_status()

    print("\n=== RÉPONSE SIMULATION ===")
    print(response.json())

    # Dans la réponse, on récupère le transaction_id créé artificiellement
    simulated = response.json()
    transaction_id = simulated.get("transaction_id") or simulated.get("id")

    print(f"\nTransaction simulée ID = {transaction_id}")

    return transaction_id


def refund_partial(transaction_id: str, original_amount: float, ratio: float = 0.8):
    """Remboursement partiel (80% par défaut)."""
    refund_amount = round(original_amount * ratio, 2)

    url = f"{BASE_URL}/v0.1/me/refund/{transaction_id}"
    headers = {
        "Authorization": f"Bearer {SUMUP_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json={"amount": refund_amount})

    print("\n=== REMBOURSEMENT PARTIEL ===")
    print(f"Montant remboursé : {refund_amount} EUR")
    print(f"HTTP {response.status_code}")

    if response.status_code == 204:
        print("✔ Remboursement 80% OK")
    else:
        print(response.text)

    return response


def refund_full(transaction_id: str):
    """Remboursement total."""
    url = f"{BASE_URL}/v0.1/me/refund/{transaction_id}"
    headers = {"Authorization": f"Bearer {SUMUP_API_KEY}"}

    response = requests.post(url, headers=headers)

    print("\n=== REMBOURSEMENT TOTAL ===")
    print(f"HTTP {response.status_code}")

    if response.status_code == 204:
        print("✔ Remboursement total OK")
    else:
        print(response.text)

    return response

In [ ]:
price = 10.00

# 1) Création du checkout
checkout = create_checkout(SUMUP_API_KEY, merchant_code, price, "Test transaction")
checkout_id = checkout.id 

In [12]:
print(f'id : {checkout_id}')

id : 5ae9fd20-43f3-4331-a09b-33ee4fd03f90


In [13]:
# 2) Simulation du paiement
transaction_id = simulate_payment(checkout_id)

HTTPError: 404 Client Error: Not Found for url: https://api.sumup.com/v0.1/checkouts/5ae9fd20-43f3-4331-a09b-33ee4fd03f90/simulate

In [ ]:
# 3) Remboursement à 80%
refund_partial(transaction_id, price)

# 4) Remboursement total
refund_full(transaction_id)